## 🧑‍💼 Employee Attrition Prediction (Pipeline)

Given *data about employees at a company*, let's try to predict whether a given employee will **leave** the company.

We will use a logistic regression model to make our predictions.

Data source: https://www.kaggle.com/datasets/HRAnalyticRepository/employee-attrition-data

### Importing Libraries

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LogisticRegression

In [2]:
data = pd.read_csv('archive/MFG10YearTerminationData.csv')
data

,EmployeeID,recorddate_key,birthdate_key,orighiredate_key,terminationdate_key,age,length_of_service,city_name,department_name,job_title,store_name,gender_short,gender_full,termreason_desc,termtype_desc,STATUS_YEAR,STATUS,BUSINESS_UNIT
0,1318,12/31/2006 0:00,1/3/1954,8/28/1989,1/1/1900,52,17,Vancouver,Executive,CEO,35,M,Male,Not Applicable,Not Applicable,2006,ACTIVE,HEADOFFICE
1,1318,12/31/2007 0:00,1/3/1954,8/28/1989,1/1/1900,53,18,Vancouver,Executive,CEO,35,M,Male,Not Applicable,Not Applicable,2007,ACTIVE,HEADOFFICE
2,1318,12/31/2008 0:00,1/3/1954,8/28/1989,1/1/1900,54,19,Vancouver,Executive,CEO,35,M,Male,Not Applicable,Not Applicable,2008,ACTIVE,HEADOFFICE
3,1318,12/31/2009 0:00,1/3/1954,8/28/1989,1/1/1900,55,20,Vancouver,Executive,CEO,35,M,Male,Not Applicable,Not Applicable,2009,ACTIVE,HEADOFFICE
4,1318,12/31/2010 0:00,1/3/1954,8/28/1989,1/1/1900,56,21,Vancouver,Executive,CEO,35,M,Male,Not Applicable,Not Applicable,2010,ACTIVE,HEADOFFICE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49648,8258,12/1/2015 0:00,5/28/1994,8/19/2013,12/30/2015,21,2,Valemount,Dairy,Dairy Person,34,M,Male,Layoff,Involuntary,2015,TERMINATED,STORES
49649,8264,8/1/2013 0:00,6/13/1994,8/27/2013,8/30/2013,19,0,Vancouver,Customer Service,Cashier,44,F,Female,Resignaton,Voluntary,2013,TERMINATED,STORES
49650,8279,12/1/2015 0:00,7/18/1994,9/15/2013,12/30/2015,21,2,White Rock,Customer Service,Cashier,39,F,Female,Layoff,Involuntary,2015,TERMINATED,STORES
49651,8296,12/1/2013 0:00,9/2/1994,10/9/2013,12/31/2013,19,0,Kelowna,Customer Service,Cashier,16,F,Female,Resignaton,Voluntary,2013,TERMINATED,STORES


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49653 entries, 0 to 49652
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   EmployeeID           49653 non-null  int64 
 1   recorddate_key       49653 non-null  object
 2   birthdate_key        49653 non-null  object
 3   orighiredate_key     49653 non-null  object
 4   terminationdate_key  49653 non-null  object
 5   age                  49653 non-null  int64 
 6   length_of_service    49653 non-null  int64 
 7   city_name            49653 non-null  object
 8   department_name      49653 non-null  object
 9   job_title            49653 non-null  object
 10  store_name           49653 non-null  int64 
 11  gender_short         49653 non-null  object
 12  gender_full          49653 non-null  object
 13  termreason_desc      49653 non-null  object
 14  termtype_desc        49653 non-null  object
 15  STATUS_YEAR          49653 non-null  int64 
 16  STAT

### Initial Preprocessing

In [4]:
df = data.copy()

In [5]:
df

,EmployeeID,recorddate_key,birthdate_key,orighiredate_key,terminationdate_key,age,length_of_service,city_name,department_name,job_title,store_name,gender_short,gender_full,termreason_desc,termtype_desc,STATUS_YEAR,STATUS,BUSINESS_UNIT
0,1318,12/31/2006 0:00,1/3/1954,8/28/1989,1/1/1900,52,17,Vancouver,Executive,CEO,35,M,Male,Not Applicable,Not Applicable,2006,ACTIVE,HEADOFFICE
1,1318,12/31/2007 0:00,1/3/1954,8/28/1989,1/1/1900,53,18,Vancouver,Executive,CEO,35,M,Male,Not Applicable,Not Applicable,2007,ACTIVE,HEADOFFICE
2,1318,12/31/2008 0:00,1/3/1954,8/28/1989,1/1/1900,54,19,Vancouver,Executive,CEO,35,M,Male,Not Applicable,Not Applicable,2008,ACTIVE,HEADOFFICE
3,1318,12/31/2009 0:00,1/3/1954,8/28/1989,1/1/1900,55,20,Vancouver,Executive,CEO,35,M,Male,Not Applicable,Not Applicable,2009,ACTIVE,HEADOFFICE
4,1318,12/31/2010 0:00,1/3/1954,8/28/1989,1/1/1900,56,21,Vancouver,Executive,CEO,35,M,Male,Not Applicable,Not Applicable,2010,ACTIVE,HEADOFFICE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49648,8258,12/1/2015 0:00,5/28/1994,8/19/2013,12/30/2015,21,2,Valemount,Dairy,Dairy Person,34,M,Male,Layoff,Involuntary,2015,TERMINATED,STORES
49649,8264,8/1/2013 0:00,6/13/1994,8/27/2013,8/30/2013,19,0,Vancouver,Customer Service,Cashier,44,F,Female,Resignaton,Voluntary,2013,TERMINATED,STORES
49650,8279,12/1/2015 0:00,7/18/1994,9/15/2013,12/30/2015,21,2,White Rock,Customer Service,Cashier,39,F,Female,Layoff,Involuntary,2015,TERMINATED,STORES
49651,8296,12/1/2013 0:00,9/2/1994,10/9/2013,12/31/2013,19,0,Kelowna,Customer Service,Cashier,16,F,Female,Resignaton,Voluntary,2013,TERMINATED,STORES


In [6]:
# Drop unnecessary columns
unneeded_columns = ['EmployeeID', 'gender_short']
df = df.drop(unneeded_columns, axis=1)

In [7]:
# Drop all columns not available before termination
df = df.drop(['terminationdate_key', 'length_of_service', 'termreason_desc', 'termtype_desc'], axis=1)

In [8]:
df

,recorddate_key,birthdate_key,orighiredate_key,age,city_name,department_name,job_title,store_name,gender_full,STATUS_YEAR,STATUS,BUSINESS_UNIT
0,12/31/2006 0:00,1/3/1954,8/28/1989,52,Vancouver,Executive,CEO,35,Male,2006,ACTIVE,HEADOFFICE
1,12/31/2007 0:00,1/3/1954,8/28/1989,53,Vancouver,Executive,CEO,35,Male,2007,ACTIVE,HEADOFFICE
2,12/31/2008 0:00,1/3/1954,8/28/1989,54,Vancouver,Executive,CEO,35,Male,2008,ACTIVE,HEADOFFICE
3,12/31/2009 0:00,1/3/1954,8/28/1989,55,Vancouver,Executive,CEO,35,Male,2009,ACTIVE,HEADOFFICE
4,12/31/2010 0:00,1/3/1954,8/28/1989,56,Vancouver,Executive,CEO,35,Male,2010,ACTIVE,HEADOFFICE
...,...,...,...,...,...,...,...,...,...,...,...,...
49648,12/1/2015 0:00,5/28/1994,8/19/2013,21,Valemount,Dairy,Dairy Person,34,Male,2015,TERMINATED,STORES
49649,8/1/2013 0:00,6/13/1994,8/27/2013,19,Vancouver,Customer Service,Cashier,44,Female,2013,TERMINATED,STORES
49650,12/1/2015 0:00,7/18/1994,9/15/2013,21,White Rock,Customer Service,Cashier,39,Female,2015,TERMINATED,STORES
49651,12/1/2013 0:00,9/2/1994,10/9/2013,19,Kelowna,Customer Service,Cashier,16,Female,2013,TERMINATED,STORES


### Building Pipeline

In [9]:
{column: len(df[column].unique()) for column in df.select_dtypes('object').columns}

{'recorddate_key': 130,
 'birthdate_key': 5342,
 'orighiredate_key': 4415,
 'city_name': 40,
 'department_name': 21,
 'job_title': 47,
 'gender_full': 2,
 'STATUS': 2,
 'BUSINESS_UNIT': 2}

In [10]:
# Classify features by type
date_features = [
    'recorddate_key',
    'birthdate_key',
    'orighiredate_key'
]

binary_features = [
    'gender_full', 
    'BUSINESS_UNIT'
]

nominal_features = [
    'city_name', 
    'department_name', 
    'job_title'
]

In [11]:
# Let's build a transformer for date columns that will extract the year, month and day features
class DateTransformer:

    def fit(self, X, y):
        return self
        
    def transform(self, X):
        for column in X.columns:
            X[column] = pd.to_datetime(X[column])
            X[column + "_year"] = X[column].apply(lambda x: x.year)
            X[column + "_month"] = X[column].apply(lambda x: x.month)
            X[column + "_day"] = X[column].apply(lambda x: x.day)
            X = X.drop(column, axis=1)
        return X

In [12]:
# Construct transformers to handle each type of feature

binary_transformer = Pipeline(steps=[
    ('ordinal', OrdinalEncoder(categories='auto'))
])

nominal_transformer = Pipeline(steps=[
    ('nominal', OneHotEncoder())
])

date_transformer = Pipeline(steps=[
    ('date', DateTransformer())
])

In [13]:
# Build a preprocessing transformer with ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('binary', binary_transformer, binary_features),
    ('nominal', nominal_transformer, nominal_features),
    ('date', date_transformer, date_features)
], sparse_threshold = 0)

In [14]:
# Build the final pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression())
])

In [15]:
# Split df into X and y
y = df['STATUS'].copy()
X = df.drop('STATUS', axis=1).copy()

In [16]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=1)

### Training

In [17]:
model.fit(X_train, y_train)

,steps,"[('preprocessor', ...), ('scaler', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('binary', ...), ('nominal', ...), ...]"
,remainder,'drop'
,sparse_threshold,0
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


### Results

In [18]:
acc = model.score(X_test, y_test)

print("Test Accuracy: {:.2f}%".format(acc * 100))

Test Accuracy: 100.00%


/home/aditya/miniconda3/envs/tfenv/lib/python3.10/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


In [25]:
sample_input = pd.DataFrame(pd.Series({
    'recorddate_key': '6/1/2006',
    'birthdate_key': '6/28/1941',
    'orighiredate_key': '1/3/1993',
    'age': 65,
    'city_name': 'Fort St John',
    'department_name': 'Dairy',
    'job_title': 'Dairy Person',
    'store_name': '12',
    'gender_full': 'Female',
    'STATUS_YEAR': 2006,
    'BUSINESS_UNIT': 'STORES'
})).T

In [26]:
sample_input

,recorddate_key,birthdate_key,orighiredate_key,age,city_name,department_name,job_title,store_name,gender_full,STATUS_YEAR,BUSINESS_UNIT
0,6/1/2006,6/28/1941,1/3/1993,65,Fort St John,Dairy,Dairy Person,12,Female,2006,STORES


In [27]:
prediction = model.predict(sample_input)

print("Model Prediction:", prediction)

Model Prediction: ['TERMINATED']


/home/aditya/miniconda3/envs/tfenv/lib/python3.10/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


In [24]:
X_test[y_test == 'TERMINATED']

,recorddate_key,birthdate_key,orighiredate_key,age,city_name,department_name,job_title,store_name,gender_full,STATUS_YEAR,BUSINESS_UNIT
48453,6/1/2006 0:00,6/28/1941,1/3/1993,65,Fort St John,Dairy,Dairy Person,12,Female,2006,STORES
48726,10/1/2009 0:00,10/29/1944,5/5/1996,65,Chilliwack,Produce,Produce Clerk,6,Female,2009,STORES
49109,5/1/2013 0:00,5/20/1948,11/25/1999,65,Burnaby,Produce,Produce Clerk,5,Female,2013,STORES
49632,10/1/2014 0:00,9/8/1993,4/2/2013,21,Victoria,Customer Service,Cashier,46,Male,2014,STORES
48591,2/1/2008 0:00,2/2/1943,8/10/1994,65,West Vancouver,Meats,Meat Cutter,38,Female,2008,STORES
...,...,...,...,...,...,...,...,...,...,...,...
49269,3/1/2006 0:00,9/18/1977,10/25/2004,28,New Westminister,Produce,Produce Clerk,20,Male,2006,STORES
48499,1/1/2009 0:00,7/17/1956,8/20/1993,52,New Westminster,Meats,Meat Cutter,21,Female,2009,STORES
49254,12/1/2014 0:00,11/28/1975,11/12/2003,39,Grand Forks,Produce,Produce Clerk,13,Female,2014,STORES
48517,4/1/2007 0:00,4/14/1942,10/20/1993,65,Fort St John,Meats,Meat Cutter,12,Female,2007,STORES
